In [7]:
import pandas as pd
import numpy as np
from langchain.agents import initialize_agent, Tool
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_classification
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
data = pd.read_csv('titanic.csv')

In [15]:
def encode_categorical_features(data):
    """Encode categorical features with one-hot encoding or label encoding based on the number of unique categories."""
    max_categories = 10
    all_encoded_cols = ""
    all_labeled_cols = ""
    try:
        # Ensure the DataFrame contains categorical columns
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns

        if categorical_cols.size > 0:
            for col in categorical_cols:
                num_unique = data[col].nunique()

                # Apply Label Encoding if the number of unique categories is too high
                if num_unique > max_categories:
                    print(f"Applying Label Encoding to '{col}' with {num_unique} unique categories.")
                    label_encoder = LabelEncoder()
                    data[col] = label_encoder.fit_transform(data[col])
                    all_labeled_cols = all_labeled_cols + " " + col

                else:
                    print(f"Applying One-Hot Encoding to '{col}' with {num_unique} unique categories.")
                    # Apply One-Hot Encoding for columns with a manageable number of categories
                    encoder = OneHotEncoder(sparse_output=False, drop='first')
                    encoded_array = encoder.fit_transform(data[[col]])
                    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out([col]))
                    data = data.drop(columns=[col])
                    data = pd.concat([data, encoded_df], axis=1)
                    all_encoded_cols = all_encoded_cols + " " + col
        else:
            print("No categorical columns to encode.")

    except Exception as e:
        print(f"Error in encode_categorical_features: {e}")

    return data


In [16]:
encoded_data = encode_categorical_features(data)

Applying One-Hot Encoding to 'Sex' with 2 unique categories.
Applying Label Encoding to 'Ticket' with 681 unique categories.
Applying Label Encoding to 'Cabin' with 147 unique categories.
Applying One-Hot Encoding to 'Embarked' with 3 unique categories.


In [17]:
encoded_data

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,108,22.0,1,0,523,7.2500,147,1.0,0.0,1.0,0.0
1,2,1,1,190,38.0,1,0,596,71.2833,81,0.0,0.0,0.0,0.0
2,3,1,3,353,26.0,0,0,669,7.9250,147,0.0,0.0,1.0,0.0
3,4,1,1,272,35.0,1,0,49,53.1000,55,0.0,0.0,1.0,0.0
4,5,0,3,15,35.0,0,0,472,8.0500,147,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,548,27.0,0,0,101,13.0000,147,1.0,0.0,1.0,0.0
887,888,1,1,303,19.0,0,0,14,30.0000,30,0.0,0.0,1.0,0.0
888,889,0,3,413,NaN,1,2,675,23.4500,147,0.0,0.0,1.0,0.0
889,890,1,1,81,26.0,0,0,8,30.0000,60,1.0,0.0,0.0,0.0


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.datasets import load_iris
import numpy as np


In [19]:
# Load a sample dataset (e.g., Iris dataset)
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Note: For real-world datasets, more preprocessing steps like scaling, encoding categorical features, etc., might be needed.


In [28]:
import pandas as pd
data_pd = pd.DataFrame(data.data, columns=data.feature_names)

In [29]:
data_pd.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [31]:
import ast

input_string = "['Survived', 'Pclass', 'Ticket', 'Fare', 'Cabin', 'Sex_male']"
# Convert the string to a list
converted_list = ast.literal_eval(input_string)

# Display the result
print(converted_list[1])


Pclass


In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")